In [1]:
import gc

import lettuce as lt
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import warnings
import numpy as np
from lettuce.unit import UnitConversion
from lettuce.util import append_axes
from lettuce.boundary import EquilibriumBoundaryPU, BounceBackBoundary, HalfwayBounceBackBoundary, FullwayBounceBackBoundary, EquilibriumOutletP, AntiBounceBackOutlet, InterpolatedBounceBackBoundary
from lettuce.flows.cylinder2D import Cylinder2D
from lettuce.flows.obstacleCylinder import ObstacleCylinder

import torch
import time
import datetime
import os
import shutil
from pyevtk.hl import imageToVTK

In [2]:
### OUTPUT SETTINGS
output_data = True

# IMPORTANT: set correct output_path, for example "/home/YourUserName/simulation_output"
output_path = "/mnt/ScratchHDD1/Max_Scratch/lbm_simulations"  # lokal HBRS
#output_path = "/home/max/Documents/lbm_simulations"  # lokal Bonn
#output_path = "/home/mbille3s/02_lbm_simulations"  # cluster HBRS

# name: if you want something specific in the dir-name
name = "cyl2D_VRAMchecker"

# choose stencil: "D3Q9" dor 2D, "D3Q15", "D3Q19", "D3Q27" for 3D
stencil_choice = "D2Q9"
# choose collision operator: "bgk, "kbc", "reg"
collision_choice = "bgk"

In [9]:
### SIMULATION PARAMETERS (and estimation of timesteps needed to reach T_target)
re = 200   # Reynoldszahl
Ma = 0.05     # Machzahl
gp_start = 8000000
setup_diameter = 1  # D_PU = char_length_pu -> this defines the PU-Reference
flow_velocity = 1  # U_PU = char_velocity_pu -> this defines the PU-Reference velocity (u_max of inflow)

# SIMULATOR settings
u_init = 0    # initial velocity field: # 0: uniform u=0, # 1: uniform u=1 or parabolic, mean amplitude u_char_lu (similar to poiseuille-flow)
perturb_init = True   # perturb initial symmetry by small sine-wave in initial velocity field -> triggers Karman-vortex street for Re > 46
lateral_walls='periodic'  # type of top/bottom boundary: 'bounceback' = frictious wall, 'periodic' = periodic boundary, 'slip' = non-frictious wall
bc_type='fwbb'  # choose algorithm for bounceback-boundaries: fullway 'fwbb', halfway 'hwbb', linear interpolated 'ibb1'

In [4]:
### (no user input) CREATE OUTPUT DIRECTORIES
if output_data:  # toggle output
    timestamp = datetime.datetime.now()
    timestamp = timestamp.strftime("%y%m%d")+"_"+timestamp.strftime("%H%M%S")

    dir_name = "/data_" + str(timestamp) + "_" + name
    os.makedirs(output_path+dir_name)

    print("dir_name: "+dir_name)
else:
    output_vtk = False

dir_name: /data_230911_161039_cyl2D_VRAMchecker


In [9]:
# max_vram = []
# t_start=time.time()
# for i in range(2000,3000):
#     try:
#         lattice = lt.Lattice(lt.D2Q9, "cuda:0", dtype=torch.float64)
#         flow = ObstacleCylinder(shape=(i,2*i),
#                             reynolds_number=re, mach_number=Ma,
#                             lattice=lattice,
#                             char_length_pu=setup_diameter,
#                             char_length_lu=gridpoints_per_diameter,
#                             char_velocity_pu=flow_velocity,
#                             lateral_walls=lateral_walls,
#                             bc_type=bc_type,
#                             perturb_init=perturb_init,
#                             u_init=u_init
#                             )
#         tau = flow.units.relaxation_parameter_lu
#         # collision
#         if collision_choice == "reg":
#             collision=lt.RegularizedCollision(lattice, tau)
#         elif collision_choice == "kbc":
#             collision=lt.KBCCollision3D(lattice,tau)
#         else: # collision_choice = "bgk":
#             collision=lt.BGKCollision(lattice, tau)
#
#         sim = lt.Simulation(flow, lattice,
#                         collision,
#                         # lt.BGKCollision(lattice, tau),
#                         # lt.RegularizedCollision(lattice, tau),
#                         # lt.KBCCollision2D(lattice,tau),
#                         lt.StandardStreaming(lattice)
#                        )
#         mlups = sim.step(1) #Simulation mit Schrittzahl n_steps
#         max_shape = str(flow.shape)
#         max_gp = str(lattice.rho(sim.f).numel())
#         max_vram.append([i,2*i*i,torch.cuda.max_memory_allocated(lattice.device)])
#     except:
#         print("max.GP:", str((i-1)*2*(i-1)), "for", str(bc_type), "for D=", str(lattice.D), ", stencil=", str(stencil_choice))
#         max_i = i
#         break
#
# t_end=time.time()
# runtime=t_end-t_start

max.GP: 14023808 for fwbb for D= 2 , stencil= D2Q9


In [16]:
# version with bisection:
max_vram = []
gp_middle = gp_start
t_start=time.time()
done = False
failed = False
i=1
gp_lower = 0
gp_upper = gp_middle
max_gp = 0
run_up = True
while not done:
    try:
        lattice = lt.Lattice(lt.D2Q9, "cuda:0", dtype=torch.float64)
        flow = ObstacleCylinder(shape=(np.sqrt(gp_middle), np.sqrt(gp_middle)),
                            reynolds_number=re, mach_number=Ma,
                            lattice=lattice,
                            char_length_pu=setup_diameter,
                            char_length_lu=int(np.sqrt(gp_middle/5)),
                            char_velocity_pu=flow_velocity,
                            lateral_walls=lateral_walls,
                            bc_type=bc_type,
                            perturb_init=perturb_init,
                            u_init=u_init
                            )
        tau = flow.units.relaxation_parameter_lu
        # collision
        if collision_choice == "reg":
            collision=lt.RegularizedCollision(lattice, tau)
        elif collision_choice == "kbc":
            collision=lt.KBCCollision3D(lattice,tau)
        else: # collision_choice = "bgk":
            collision=lt.BGKCollision(lattice, tau)

        sim = lt.Simulation(flow, lattice,
                        collision,
                        # lt.BGKCollision(lattice, tau),
                        # lt.RegularizedCollision(lattice, tau),
                        # lt.KBCCollision2D(lattice,tau),
                        lt.StandardStreaming(lattice)
                       )
        mlups = sim.step(1)
        max_shape = str(flow.shape)
        max_gp = str(lattice.rho(sim.f).numel())
        max_vram.append([i,max_gp,torch.cuda.max_memory_allocated(lattice.device)])
        failed = False
        try:
            del lattice
        except:
            pass
        try:
            del flow
        except:
            pass
        try:
            del sim
        except:
            pass
        try:
            del collision
        except:
            pass
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats(device="cuda:0")
    except:
        #print("failed for gp_middle =", str(gp_middle), "for", str(bc_type), "for D=", str(lattice.D), ", stencil=", str(stencil_choice))
        failed = True
        run_up = False
        try:
            del lattice
        except:
            pass
        try:
            del flow
        except:
            pass
        try:
            del sim
        except:
            pass
        try:
            del collision
        except:
            pass
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats(device="cuda:0")
    if failed:
        status = "failed"
    else:
        status = "success"
    print(str(i) + ": " + status + " for gp = {:e} in [{:e}, {:e}]".format(gp_middle,gp_lower,gp_upper))

    if run_up and not failed:
        gp_lower = gp_middle
        gp_middle = 2*gp_middle
        gp_upper = gp_middle
    else:  # if not run_up, do bisection
        if not failed:
            gp_lower = gp_middle
        else:  # if failed
            gp_upper = gp_middle
            if (gp_upper-gp_lower)/gp_lower < 0.05:  # limit found
                done = True
        gp_middle = int((gp_upper+gp_lower)/2)
    i += 1

t_end=time.time()
runtime=t_end-t_start

print("FINISHED BISECTION")
print("FOUND LIMIT: " + str(max_gp))
print("runtime = " + str(runtime) + " seconds")

1: success for gp = 8.000000e+06 in [0.000000e+00, 8.000000e+06]
failed for gp_middle = 16000000 for fwbb for D= 2 , stencil= D2Q9
2: failed for gp = 1.600000e+07 in [8.000000e+06, 1.600000e+07]
3: success for gp = 1.200000e+07 in [8.000000e+06, 1.600000e+07]
4: success for gp = 1.400000e+07 in [1.200000e+07, 1.600000e+07]
5: success for gp = 1.500000e+07 in [1.400000e+07, 1.600000e+07]
6: success for gp = 1.550000e+07 in [1.500000e+07, 1.600000e+07]
failed for gp_middle = 15750000 for fwbb for D= 2 , stencil= D2Q9
7: failed for gp = 1.575000e+07 in [1.550000e+07, 1.600000e+07]
FINISHED BISECTION
FOUND LIMIT: 15499969
runtime = 171.5482771396637 seconds


In [12]:
gc.collect()


14

In [14]:
print(max_vram)

[[1, 8000000, 3698561536], [3, 12000000, 4955796992], [4, 14000000, 5780452352], [5, 15000000, 6194767872]]


In [19]:
# output data checker
if output_data:
    output_file = open(output_path+dir_name+"/"+timestamp + "_parms_stats_obs.txt", "a")
    output_file.write("DATA for "+timestamp)
    output_file.write("\n\n###   Parameters   ###")
    output_file.write("\nbc_type = "+str(bc_type))
    output_file.write("\nlateral_walls = "+str(lateral_walls))
    output_file.write("\nstencil = "+str(stencil_choice))
    output_file.write("\ncollision = " + str(collision_choice))
    output_file.write("\n")
    output_file.write("\nperturb_init = " + str(perturb_init))
    output_file.write("\n")

    output_file.write("\n\n###   STATS  ###")
    output_file.write("\nlargest shape: " + max_shape)
    output_file.write("\ni = " + str(i))
    output_file.write("\nmax. GP = " + max_gp)
    output_file.write("\nmax. VRAM = " + str(max_vram[-1][2]))
    output_file.write("\ntotal runtime VRAM_check = " + str(runtime))
    output_file.close()


In [20]:
# output data for all i
if output_data:
    output_file = open(output_path+dir_name+"/"+timestamp + "_i_gp_vram.txt", "a")
    for i in range(len(max_vram)):
        output_file.write(str(max_vram[i][0]) + ", " + str(max_vram[i][1]) + ", " + str(max_vram[i][2]) + "\n")
    output_file.close()